In [ ]:
import random
import sympy as sp

def random_boolean_formula(variables, depth=3, prob_literal=0.3, prob_const=0.1):
    """Генерирует случайную булеву формулу."""
    if depth == 0 or (random.random() < prob_literal and depth > 1):
        if random.random() < prob_const:
            return random.choice([sp.true, sp.false])
        var = random.choice(variables)
        return var if random.random() > 0.5 else ~var

    left = random_boolean_formula(variables, depth - 1, prob_literal, prob_const)
    right = random_boolean_formula(variables, depth - 1, prob_literal, prob_const)
    op = random.choice([sp.And, sp.Or])
    return op(left, right)

variables = [sp.Symbol("x"), sp.Symbol("y"), sp.Symbol("z")]
formula = random_boolean_formula(variables, depth=4)
print(formula)

True


In [ ]:
def formula_to_edtl(name, formula):
    """Преобразует булеву формулу в формат EDTL."""
    return f"    {name} := {formula_to_string(formula, top_level=True)};"

def formula_to_string(expr, top_level=False):
    """Преобразует sympy-формулу в строку, убирая ненужные скобки."""
    if expr == sp.true:
        return "TRUE"
    if expr == sp.false:
        return "FALSE"
    if isinstance(expr, sp.Symbol):
        return str(expr)
    if isinstance(expr, sp.Not):
        inner = formula_to_string(expr.args[0], top_level=False)
        return f"NOT {inner}"

    if isinstance(expr, (sp.And, sp.Or)):
        op = " AND " if isinstance(expr, sp.And) else " OR " if isinstance(expr, sp.Or) else " XOR "
        args = [formula_to_string(arg, top_level=False) for arg in expr.args]

        return op.join(args) if top_level else f"({op.join(args)})"

    return str(expr)

def generate_random_req(req_name, variables):
    """Генерирует случайное EDTL-требование."""
    req_parts = {
        "TRIGGER": random_boolean_formula(variables),
        "RELEASE": random_boolean_formula(variables),
        "DELAY": random_boolean_formula(variables),
        "FINAL": random_boolean_formula(variables),
        "REACTION": random_boolean_formula(variables),
        "INVARIANT": random_boolean_formula(variables)
    }

    edtl_req = f"REQ {req_name}\n"
    edtl_req += "\n".join(formula_to_edtl(name, expr) for name, expr in req_parts.items())
    edtl_req += "\nEND_REQ"

    return edtl_req


variables = [sp.Symbol("H"), sp.Symbol("D"), sp.Symbol("G")]
print(generate_random_req("THIRD", variables))

REQ THIRD
    TRIGGER := NOT H OR (G AND NOT G) OR (H AND NOT D);
    RELEASE := H OR (G AND NOT G) OR (NOT D AND NOT H);
    DELAY := G OR H OR (H AND NOT D);
    FINAL := H;
    REACTION := NOT G;
    INVARIANT := (NOT D OR NOT G) AND (D OR NOT G OR (NOT D AND NOT H));
END_REQ


In [ ]:
from sympy.logic import satisfiable
def is_tautology(expr):
  return satisfiable(expr.simplify()) == {True: True}


x = sp.Symbol('x')
is_tautology(x | ~x)

True

In [ ]:
def generate_req_parts(variables):
    req_parts = {
        "TRIGGER": random_boolean_formula(variables),
        "RELEASE": random_boolean_formula(variables),
        "DELAY": random_boolean_formula(variables),
        "FINAL": random_boolean_formula(variables),
        "REACTION": random_boolean_formula(variables),
        "INVARIANT": random_boolean_formula(variables)
    }
    return req_parts

def check_conditions(r1, r2, conditions, variables):
  if is_tautology(r1["TRIGGER"] >> r2["TRIGGER"]):
    rs1, rs2 = r1, r2
  elif is_tautology(r2["TRIGGER"] >> r1["TRIGGER"]):
    rs1, rs2 = r2, r1
  else:
    return False

  if is_tautology(conditions(rs1, rs2)):
    return True
  return False


def generate_ltl_pair(conditions, variables):
  r1 = generate_req_parts(variables)
  r2 = generate_req_parts(variables)
  while not check_conditions(r1, r2, conditions, variables) :
    r1 = generate_req_parts(variables)
    r2 = generate_req_parts(variables)
  return r1, r2

TRG = lambda r1, r2: (r1["TRIGGER"] & ~r1["RELEASE"]) | r1["INVARIANT"] | (r2["TRIGGER"] & ~r2["RELEASE"])

conditions = lambda r1, r2: ~(r1["INVARIANT"] & r2["INVARIANT"]) & ~(TRG(r1, r2) >> (r1["FINAL"] & r1["REACTION"])) & ~(TRG(r1, r2) >> (r2["FINAL"] & r2["REACTION"]))
generate_ltl_pair(conditions, variables)

({'TRIGGER': D | H | ~D | (D & ~D) | (G & ~D),
  'RELEASE': False,
  'DELAY': D,
  'FINAL': H & (D | ~D | (~D & ~G)),
  'REACTION': ~H & ((D & H) | (D & ~D)),
  'INVARIANT': ~D},
 {'TRIGGER': H & (H | ~G),
  'RELEASE': G | ~D | ~G | (D & ~G),
  'DELAY': H,
  'FINAL': False,
  'REACTION': D | G | H | ~H | (D & G & ~D & ~G),
  'INVARIANT': D})

In [ ]:
def generate_edtl_pair(conditions, variables):
    """Генерирует случайное EDTL-требование."""
    r1, r2 = generate_ltl_pair(conditions, variables)

    edtl_reqs = "VAR_INPUT\n"
    edtl_reqs += "\n".join(f"    {var} : BOOL;" for var in variables)
    edtl_reqs += "\nEND_VAR\n\n"

    edtl_reqs += "REQ FIRST\n"
    edtl_reqs += "\n".join(formula_to_edtl(name, expr) for name, expr in r1.items())
    edtl_reqs += "\nEND_REQ\n\n"

    edtl_reqs += "REQ SECOND\n"
    edtl_reqs += "\n".join(formula_to_edtl(name, expr) for name, expr in r2.items())
    edtl_reqs += "\nEND_REQ\n\n"

    return edtl_reqs

print(generate_edtl_pair(conditions, variables))

VAR_INPUT
    H : BOOL;
    D : BOOL;
    G : BOOL;
END_VAR

REQ FIRST
    TRIGGER := NOT G OR ((G OR NOT H) AND (NOT D OR NOT G));
    RELEASE := G OR NOT H OR (G AND NOT G);
    DELAY := NOT D OR (D AND H);
    FINAL := G AND H AND NOT H AND (H OR NOT G);
    REACTION := G;
    INVARIANT := G;
END_REQ

REQ SECOND
    TRIGGER := (H AND NOT G AND (G OR NOT G)) OR ((D OR NOT D) AND (H OR NOT H));
    RELEASE := H;
    DELAY := H OR NOT D OR (G AND NOT H) OR (H AND NOT H);
    FINAL := NOT H;
    REACTION := (D AND H) OR (D AND NOT D) OR (H AND NOT H);
    INVARIANT := FALSE;
END_REQ




In [ ]:
TRG = lambda r1, r2: (r1["TRIGGER"] & ~r1["RELEASE"]) | r1["INVARIANT"] | (r2["TRIGGER"] & ~r2["RELEASE"])

conditions1 = lambda r1, r2: ~(r1["INVARIANT"] & r2["INVARIANT"]) & ~(TRG(r1, r2) >> (r1["FINAL"] & r1["REACTION"])) & ~(TRG(r1, r2) >> (r2["FINAL"] & r2["REACTION"]))

conditions2 = lambda r1, r2: ~(r1["RELEASE"] & r2["INVARIANT"]) & ~(TRG(r1, r2) >> (r2["FINAL"] & r2["REACTION"]))

conditions3 = lambda r1, r2: ~((r1["FINAL"] & r1["REACTION"]) & r2["INVARIANT"]) & ~(TRG(r1, r2) >> (r2["FINAL"] & r2["REACTION"]))

conditions4 = lambda r1, r2: (r1["RELEASE"] >> r2["FINAL"]) & ~(r1["RELEASE"] & r2["INVARIANT"]) & ~((r1["RELEASE"] | r2["FINAL"]) >> (r2["RELEASE"] | r2["REACTION"]))

conditions5 = lambda r1, r2: (r1["REACTION"] >> r2["FINAL"]) & ~(r1["REACTION"] & r2["INVARIANT"]) & ~((r1["REACTION"] | r2["FINAL"]) >> (r2["RELEASE"] | r2["REACTION"]))

conditions6 = lambda r1, r2: (r1["FINAL"] >> r2["FINAL"]) & ~(r1["FINAL"] & r2["INVARIANT"]) & ~((r1["FINAL"] | r2["FINAL"]) >> (r2["RELEASE"] | r2["REACTION"]))

conditions7 = lambda r1, r2: (r1["FINAL"] >> r2["FINAL"]) & ~(r1["INVARIANT"] & ~r2["DELAY"]) & ~((r1["INVARIANT"] | r1["FINAL"]) & r2["RELEASE"]) & ~((r1["INVARIANT"] | r1["FINAL"]) & r2["REACTION"])

conditions_no = lambda r1, r2: not is_tautology(conditions1(r1, r2)) and not is_conditions2(r1, r2) and not is_tautology(conditions3(r1, r2)) and not is_tautology(conditions4(r1, r2)) and not is_tautology(conditions5(r1, r2)) and not is_tautology(conditions6(r1, r2)) and not is_tautology(conditions7(r1, r2))


In [ ]:
import random
import string

def random_variables(min_size=2, max_size=10):
    num_letters = random.randint(min_size, max_size)
    letters = random.sample(sp.symbols('A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U,V,W,X,Y,Z'), num_letters)
    return letters

variables = random_variables()
print(variables)

[V, E, X]


In [ ]:
import os
import random
import string
import zipfile

conditions = [conditions1, conditions2, conditions3, conditions4, conditions5, conditions6, conditions7, conditions_no]

folder_name = "edtl_tests"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

for idx, condition in enumerate(conditions, start=1):
    for i in range(1, 51):
      filename = os.path.join(folder_name, f"reqs_{idx}_{i:03d}.edtl")

      if not os.path.exists(filename):
        text = generate_edtl_pair(condition, random_variables())
        with open(filename, 'w', encoding='utf-8') as f:
          f.write(text)
    print(idx)


zip_filename = "reqs.zip"
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file in os.listdir(folder_name):
        zipf.write(os.path.join(folder_name, file), file)

print(f"Создан ZIP-архив: {zip_filename}")

1
2
3
4
5
6
7
Создан ZIP-архив: reqs.zip


In [ ]:
import shutil
shutil.rmtree(folder_name)
print(f"Временная папка {folder_name} удалена")

Временная папка edtl_tests удалена
